<h1 style="text-align: center;">AAG - Méthodes Classiques en IA</h1>


# Naive Bayes

## Préparation du dataset & création des corpus

**<ins>Dataset R6 :</ins>**
<style>
    .heatMap {
        width: 70%;
        text-align: center;
    }
    .heatMap th {
        background: grey;
        word-wrap: break-word;
        text-align: center;
    }

</style>

<ins>Types de features</ins>
<div class="heatMap">
									

|   Nom des features           |  Type       | description |
|:---------------:|:-----------:|:-----------:|
|   skillrank  |   Nom (string)   | rang du joueur |
|   operator  |   Nom (string)  | nom du personnage |
|   primaryweapon   |   Nom (string)   | nom de l'arme principale |
|   secondaryweapon   |   nom (string)   | nom de l'arme secondaire |
|   secondarygadget   |   nom (string)   | nom du gadget utilisé |
|   k/d   |   numeric (int)   | ratio mort/kill (1 si >1 sinon 0) |
|   target   |   numeric (int)   | ratio victoire/partie (1 si >50% sinon 0)
</div>

<ins>Séparation du corpus</ins>
<div class="heatMap">

|                 | Train (2/3) | Test(1/3) | Total / type de w/r |
|:---------------:|:-----:|:----:|:------:|
| 1 (w/r positif)  |   398  |  199  |   597   |
| 0 (w/r negatif) |   1035  |  518  |   1553   |
| Total / corpus |   1433 | 717  |  2150   |

</div>

on peut utiliser le naive bayes car nos descripteurs sont tous discrétisé / discrets

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Chargement du dataset dans une DataFrame
dataset_R6 = pd.read_csv("./DatasetR6/S5_operators.csv", sep=';')

# on calcule le w/r
dataset_R6.insert(dataset_R6.shape[1],"w/r", (dataset_R6["nbwins"]/dataset_R6["nbpicks"]), True) 
dataset_R6.drop(columns=["nbwins","nbpicks","role"], inplace=True)

# on calcule le k/d
dataset_R6.loc[dataset_R6['nbdeaths'] == 0] = 1
# on a une ligne qui fait que des 1 partout 
dataset_R6.drop(axis=0, index=dataset_R6[dataset_R6['operator'] == 1].index, inplace=True)
dataset_R6['k/d'] = dataset_R6['nbkills'] / dataset_R6['nbdeaths']


# on merge et enleve les colonnes plateformes et dateid sans perdre les données associées
dataset_R6 = dataset_R6.groupby(['skillrank','operator', 'primaryweapon', 'secondaryweapon', 'secondarygadget'], as_index=False)[['w/r', 'k/d']].mean()

# discretisation de w/r
dataset_R6.insert(dataset_R6.shape[1],"target", (dataset_R6["w/r"]>=0.5).astype(int), True) 

dataset_R6.drop(columns=["w/r"], inplace=True)

#discrétisation de k/d
dataset_R6['k/d'] = (dataset_R6["k/d"]>=1.0).astype(int)


# changement de la taille d'affichage des dataframes
#pd.set_option('display.max_rows', dataset_R6.shape[0]+1)


# création des corpus annotés et non annotés
corpus_annot = dataset_R6.copy()
corpus_annot = corpus_annot[['target']]

corpus_non_annot = dataset_R6.copy()
corpus_non_annot.drop(columns=["target"], inplace=True)

x_train, x_test, y_train, y_test = train_test_split(corpus_non_annot, corpus_annot, test_size=1/3, random_state=42, stratify=corpus_annot)

corpus_test_non_annot = x_test.copy()

corpus_train = x_train.copy()
corpus_train['target'] = y_train.copy()

corpus_verif = x_test.copy()
corpus_verif['target'] = y_test.copy()

corpus_verif


,skillrank,operator,primaryweapon,secondaryweapon,secondarygadget,k/d,target
583,Copper,SWAT-PULSE,M1014,M45 MEUSOC,NITRO CELL,0,0
962,Gold,GIGN-RESERVE,F2,LFP586,FRAG GRENADE,0,0
992,Gold,GIGN-ROOK,P90,P9,DEPLOYABLE SHIELD,0,1
1612,Silver,GIGN-ROOK,SG-CQB,P9,DEPLOYABLE SHIELD,0,1
368,Copper,GIGN-RESERVE,SG-CQB,LFP586,DEPLOYABLE SHIELD,0,0
211,Bronze,SPETSNAZ-FUZE,AK-12,PMM,STUN GRENADE,1,0
1199,Gold,SWAT-RESERVE,556XI,5.7 USG,STUN GRENADE,0,0
2122,Unranked,SWAT-RESERVE,556XI,5.7 USG,FRAG GRENADE,0,0
1997,Unranked,SAS-RESERVE,M590A1,P226 MK 25,FRAG GRENADE,0,0
43,Bronze,GIGN-MONTAGNE,Shield,P9,STUN GRENADE,0,0


 ## Préparation de vraisemblance / proba à priori

In [63]:
def calcul_vraisemblance(nom_attribut, valeur_attribut, valeur_target):
    corpus_vraisemblance = corpus_verif.copy().groupby([nom_attribut,'target'], as_index=False).size()
    #nombre de fois que l'attribut apparait dans le corpus avec ce target
    nb_val_targ = corpus_vraisemblance[(corpus_vraisemblance[nom_attribut] == valeur_attribut) & (corpus_vraisemblance['target'] == valeur_target)]['size'].sum()
    
    #nombre de fois que ce target apparait dans le corpus
    nb_val_total = corpus_vraisemblance[corpus_vraisemblance['target'] == valeur_target]['size'].sum()
    return  nb_val_targ / nb_val_total

def calcul_a_priori(valeur_target):
    corpus_a_priori = corpus_verif.copy().groupby(['target'], as_index=False).size()
    nb_target = corpus_a_priori[corpus_a_priori['target'] == valeur_target]['size'].sum()
    nb_autre = corpus_a_priori[corpus_a_priori['target'] != valeur_target]['size'].sum()
    return nb_target / (nb_autre + nb_target)

print(calcul_vraisemblance('skillrank', 'Bronze', 0))
calcul_a_priori(0)

0.138996138996139


0.7224546722454672

On va construire un dictionaire des vraisemblance avec le corpus d'entraînement

In [64]:
def train_naive_bayes(df_train):
    dict_vraisemblance = {}

    for nom_attribut in df_train:
        if(nom_attribut != 'target'):
            dict_vraisemblance[nom_attribut] = {}
            for valeurs in df_train[nom_attribut].unique():
                dict_vraisemblance[nom_attribut][valeurs] = {}
                for classe in df_train['target'].unique():
                    dict_vraisemblance[nom_attribut][valeurs][classe] = calcul_vraisemblance(nom_attribut, valeurs, classe)
                    
    return dict_vraisemblance

dict_vrais = train_naive_bayes(corpus_train)
dict_vrais

{'skillrank': {'Bronze': {0: 0.138996138996139, 1: 0.1306532663316583},
  'Platinum': {0: 0.12162162162162163, 1: 0.24623115577889448},
  'Unranked': {0: 0.21428571428571427, 1: 0.005025125628140704},
  'Silver': {0: 0.13320463320463322, 1: 0.1658291457286432},
  'Diamond': {0: 0.09266409266409266, 1: 0.20603015075376885},
  'Copper': {0: 0.16988416988416988, 1: 0.04020100502512563},
  'Gold': {0: 0.12934362934362933, 1: 0.20603015075376885}},
 'operator': {'GIGN-DOC': {0: 0.032818532818532815, 1: 0.06030150753768844},
  'SAT-ECHO': {0: 0.013513513513513514, 1: 0.035175879396984924},
  'SPETSNAZ-RESERVE': {0: 0.08494208494208494, 1: 0.01507537688442211},
  'GIGN-RESERVE': {0: 0.10617760617760617, 1: 0.005025125628140704},
  'SWAT-PULSE': {0: 0.02895752895752896, 1: 0.06532663316582915},
  'SWAT-RESERVE': {0: 0.08301158301158301, 1: 0.005025125628140704},
  'JTF2-BUCK': {0: 0.023166023166023165, 1: 0.005025125628140704},
  'SPETSNAZ-FUZE': {0: 0.04633204633204633, 1: 0.05025125628140704

## Hypothèse

In [65]:
def hypothese_naive_bayes_serie(data, dict_vraisemblance):
    meilleur_proba = -1
    meilleur_target = -1
    for target in range(0,2):
        proba_posteriori = np.longdouble(1)
        
        for nom_attribut in data.keys():
            proba_posteriori *= dict_vraisemblance[nom_attribut][data[nom_attribut]][target]
        proba_posteriori *= calcul_a_priori(target)
        
        if(proba_posteriori > meilleur_proba):
            meilleur_proba = proba_posteriori
            meilleur_target = target
            
    return meilleur_target

hypothese_naive_bayes_serie(corpus_test_non_annot.iloc[20], dict_vrais)

1

In [66]:

pd.set_option('display.max_rows', corpus_test_non_annot.shape[0]+1)

def hypothese_naive_bayes(dataframe, dict_vraisemblance):
    #copie pour éviter d'étiqueter le dataframe original
    copie_df = dataframe.copy()
    target = []
    
    for _, row in copie_df.iterrows():
        target.append(hypothese_naive_bayes_serie(row, dict_vraisemblance))
        
    # étiquetage du dataframe
    copie_df['target'] = target
    return copie_df
  
hypothese_naive_bayes(corpus_test_non_annot, dict_vrais).groupby(['target'], as_index=False).size()

,target,size
0,0,523
1,1,194


Petite prédiction pour le fun :

un rook gold avec un stuff basique mais qui possède un k/d positif 

In [67]:
dict = {
  "skillrank": "Gold",
  "operator": "GIGN-ROOK",
  "primaryweapon": 'P90',
  "secondaryweapon": 'P9',
  "secondarygadget": 'IMPACT GRENADE',
  "k/d": 0
  }

data = pd.Series(dict)
hypothese_naive_bayes_serie(data, dict_vrais)

1

On fait l'hypothèse qu'il aura un winrate positif